In [3]:
import timeit
import numpy as np
import networkx as nx

from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler import CouplingMap
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeAuckland
from qiskit.quantum_info import Pauli, SparsePauliOp



def get_operator(G):
    weight_matrix = nx.adjacency_matrix(G).todense()

    num_nodes = len(weight_matrix)
    pauli_list = []
    coeffs = []
    shift = 0

    for i in range(num_nodes):
        for j in range(i):
            if weight_matrix[i, j] != 0:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(-0.5)
                shift += 0.5

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(1.0)
            else:
                shift += 1

    return SparsePauliOp(pauli_list, coeffs=coeffs), shift

# def test_function(n, reps=5, seed = 100, method ='statevector', device=None):

#     np.random.seed(seed)                                                                     
#     deg = 2
#     G = nx.random_regular_graph(deg, n, seed = seed)

#     qubit_op, _ = get_operator(G)
#     cm = CouplingMap().from_full(n)

#     backend = AerSimulator(method=method, device=device, coupling_map=cm)

#     circuit = QAOAAnsatz(cost_operator=qubit_op, reps=reps)
#     num_parameters = circuit.num_parameters
#     circuit = circuit.assign_parameters(np.random.rand(num_parameters))
#     circuit.save_state()
#     circuit = transpile(circuit, backend=backend, coupling_map=cm, seed_transpiler=seed)

#     start = timeit.default_timer()
#     result = backend.run(circuit).result()
#     end = timeit.default_timer()

#     return end-start

def test_function(n, reps=5, seed = 100, method ='density_matrix', device=None):

    np.random.seed(seed)                                                                       
    deg = 2
    G = nx.random_regular_graph(deg, n, seed = seed)
    qubit_op, _ = get_operator(G)

    model = FakeAuckland()
    noise_model = NoiseModel.from_backend(model)
    cm = model.coupling_map
    basis_gates = noise_model.basis_gates

    backend = AerSimulator(noise_model=noise_model,
                       coupling_map=cm,
                       basis_gates=basis_gates,
                       method=method,
                       device=device)

    circuit = QAOAAnsatz(cost_operator=qubit_op, reps=reps)
    num_parameters = circuit.num_parameters
    circuit = circuit.assign_parameters(np.random.rand(num_parameters))
    circuit.save_state()

    circuit = transpile(circuit, backend=backend, seed_transpiler=seed, coupling_map=cm)

    start = timeit.default_timer()
    result = backend.run(circuit).result()
    end = timeit.default_timer()

    return end-start